In [ ]:
import json 
import redis
import cherrypy

seriesName = "StutteringEvent"

In [ ]:
#open redis connection
REDIS_HOST = "redis-11915.c98.us-east-1-4.ec2.cloud.redislabs.com"
REDIS_PORT = 11915
REDIS_USER = "default"
REDIS_PASSWORD = "kn1Ktpz2ObSEwX7s1lA9KttX9IYgy5yC"

redis_client = redis.Redis(host = REDIS_HOST, port = REDIS_PORT, password = REDIS_PASSWORD, username= REDIS_USER)
print("Is connected:", redis_client.ping())


Is connected: True


In [ ]:
class DeleteSensorHistory:
    """Either deletes the history a single sensor or all sensors history depending on the path"""
    exposed = True

    def DELETE(self, *path, **query):
        if len(path) == 0:
            sensorList = [name.decode().split(":")[0] for name in redis_client.keys(f"*:{seriesName}")]
            for sensor in sensorList:
                redis_client.delete(f"{sensor}:{seriesName}")
        else:
                redis_client.delete(f"{path[0]}:{seriesName}")

        
class ListSensors:
    """List all sensor names"""
    exposed = True

    def GET(self, *path, **query):
        sensorList = [name.decode().split(":")[0] for name in redis_client.keys(f"*:{seriesName}")]
        output_dict = {"sensor" : sensorList}
        return json.dumps(output_dict)


class SensorHistory:
    """If no path and no query then returns all the sensors histories
       If only path and no query then returns the specific senso's history
       If only query and no path then return all sensors' history in the specific range
       If both path and query returns the spefic sensor's history in the specific range"""

    exposed = True

    def GET(self, *path, **query):
        output_dict = {}

        #Check for query parameters 
        if query.get("from") is None:
            start = "-"
        else:
            start = query.get("from")
        if query.get("to") is None:
            end = "+"
        else:
            end = query.get("to")

        #If any sensor is provided
        if len(path) == 0:
            sensorList = [name.decode().split(":")[0] for name in redis_client.keys(f"*:{seriesName}")]
            for sensor in sensorList:
                stutterEvents = redis_client.ts().range(f"{sensor}:{seriesName}", start, end)
                output_dict[sensor] = {
                    "timestamps" : [ts for ts, bl in stutterEvents],
                    "stutteringEvents" : [bl for ts, bl in stutterEvents]
                }

        #If a sensor is provided
        else:
            sensor = path[0]
            if len(redis_client.keys(f"{sensor}:*"))==0:
                raise cherrypy.HTTPError(404, 'Not Found: invalid sensor name.')
            else: 
                stutterEvents = redis_client.ts().range(f"{sensor}:{seriesName}", start, end)
                output_dict[sensor] = {
                    "timestamps" : [ts for ts, bl in stutterEvents],
                    "stutteringEvents" : [bl for ts, bl in stutterEvents]
                }

        return json.dumps(output_dict)


In [ ]:
if __name__== '__main__':
    conf = {
        '/': {'request.dispatch': cherrypy.dispatch.MethodDispatcher()}
    }

    #create link between the class and the endpoint
    cherrypy.tree.mount(DeleteSensorHistory(), '/delete', conf)
    cherrypy.tree.mount(ListSensors(), '/list', conf)
    cherrypy.tree.mount(SensorHistory(), '/sensor', conf)
  
    #specify the IP address of the host and host
    cherrypy.config.update({'server.socket_host': '0.0.0.0'})
    cherrypy.config.update({'server.socket_port': 8080}) 

    #start and keep it online
    cherrypy.engine.start()
    cherrypy.engine.block()

[06/Jul/2023:14:31:37] ENGINE Bus STARTING
[06/Jul/2023:14:31:37] ENGINE Started monitor thread 'Autoreloader'.
[06/Jul/2023:14:31:38] ENGINE Serving on http://0.0.0.0:8080
[06/Jul/2023:14:31:38] ENGINE Bus STARTED
172.3.5.240 - - [06/Jul/2023:14:31:59] "GET /sensor?from=0&to=1688653919557 HTTP/1.1" 200 598 "" "python-requests/2.28.1"
172.3.135.27 - - [06/Jul/2023:14:31:59] "GET /sensor?from=0&to=1688653919676 HTTP/1.1" 200 598 "" "python-requests/2.28.1"
172.3.135.27 - - [06/Jul/2023:14:32:00] "GET /list HTTP/1.1" 200 110 "" "python-requests/2.28.1"
172.3.24.25 - - [06/Jul/2023:14:32:00] "GET /sensor?from=0&to=1688653920219 HTTP/1.1" 200 598 "" "python-requests/2.28.1"
172.3.24.25 - - [06/Jul/2023:14:32:00] "GET /sensor?from=0&to=1688653920295 HTTP/1.1" 200 598 "" "python-requests/2.28.1"
[06/Jul/2023:16:49:29] ENGINE Keyboard Interrupt: shutting down bus
[06/Jul/2023:16:49:29] ENGINE Bus STOPPING
[06/Jul/2023:16:49:29] ENGINE HTTP Server cherrypy._cpwsgi_server.CPWSGIServer(('0.0.0.0

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=5c74a7a1-d505-4b2b-a4fa-efea0c0f20da' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>